## Parts of Speech (POS) tagging

In [1]:
import spacy
import pandas as pd

# !python -m spacy download en_core_web_sm

In [2]:
nlp = spacy.load("en_core_web_sm") 

In [3]:
emma_ja = "emma woodhouse handsome clever and rich with a comfortable home and happy disposition seemed to unite some of the best blessings of existence and had lived nearly twentyone years in the world with very little to distress or vex her she was the youngest of the two daughters of a most affectionate indulgent father and had in consequence of her sisters marriage been mistress of his house from a very early period her mother had died too long ago for her to have more than an indistinct remembrance of her caresses and her place had been supplied by an excellent woman as governess who had fallen little short of a mother in affection sixteen years had miss taylor been in mr woodhouses family less as a governess than a friend very fond of both daughters but particularly of emma between them it was more the intimacy of sisters even before miss taylor had ceased to hold the nominal office of governess the mildness of her temper had hardly allowed her to impose any restraint and the shadow of authority being now long passed away they had been living together as friend and friend very mutually attached and emma doing just what she liked highly esteeming miss taylors judgment but directed chiefly by her own"
print(emma_ja)

emma woodhouse handsome clever and rich with a comfortable home and happy disposition seemed to unite some of the best blessings of existence and had lived nearly twentyone years in the world with very little to distress or vex her she was the youngest of the two daughters of a most affectionate indulgent father and had in consequence of her sisters marriage been mistress of his house from a very early period her mother had died too long ago for her to have more than an indistinct remembrance of her caresses and her place had been supplied by an excellent woman as governess who had fallen little short of a mother in affection sixteen years had miss taylor been in mr woodhouses family less as a governess than a friend very fond of both daughters but particularly of emma between them it was more the intimacy of sisters even before miss taylor had ceased to hold the nominal office of governess the mildness of her temper had hardly allowed her to impose any restraint and the shadow of auth

In [4]:
spacy_doc = nlp(emma_ja)
# spacy_doc is a container with tokens and annotations (like words, POS tags, dependencies, entities).
# print(spacy_doc[0].text, spacy_doc[0].pos_) 

pos_df = pd.DataFrame(columns=['token','pos_tag'])

for token in spacy_doc:
    row_dict = {'token': token.text, 'pos_tag': token.pos_}

    row_df = pd.DataFrame([row_dict])

    pos_df = pd.concat([pos_df, row_df], ignore_index=True)

pos_df.head(10)

,token,pos_tag
0,emma,PROPN
1,woodhouse,PROPN
2,handsome,ADJ
3,clever,ADJ
4,and,CCONJ
5,rich,ADJ
6,with,ADP
7,a,DET
8,comfortable,ADJ
9,home,NOUN


In [12]:
# Group by token and pos_tag (unique word + POS combination)
grouped = pos_df.groupby(['token', 'pos_tag'])

# Count how many times each group occurs
counts = grouped.size()

# Convert the result into a DataFrame and name the count column as 'counts'
counts_df = counts.reset_index(name='counts')

# Sort the DataFrame so that the most frequent tokens come first
pos_df_counts = counts_df.sort_values(by='counts', ascending=False)

pos_df_counts.head(10)

,token,pos_tag,counts
88,of,ADP,14
54,her,PRON,9
49,had,AUX,9
6,and,CCONJ,8
111,the,DET,8
0,a,DET,6
114,to,PART,5
61,in,ADP,4
120,very,ADV,4
13,been,AUX,4


In [7]:
# counts of pos_tags
pos_df_poscounts = pos_df_counts.groupby(['pos_tag'])['token'].count().sort_values(ascending=False)
pos_df_poscounts.head(10)

pos_tag
NOUN     35
VERB     19
ADJ      18
ADV      18
PRON      9
ADP       8
PROPN     6
DET       5
AUX       4
CCONJ     3
Name: token, dtype: int64

In [14]:
nouns = pos_df_counts[pos_df_counts.pos_tag == "NOUN"][0:10]
nouns

,token,pos_tag,counts
48,governess,NOUN,3
46,friend,NOUN,3
130,years,NOUN,2
28,daughters,NOUN,2
35,emma,NOUN,2
82,mother,NOUN,2
103,sisters,NOUN,2
1,affection,NOUN,1
11,authority,NOUN,1
32,distress,NOUN,1


## Name Entity Relation (NER)

In [16]:
import spacy
from spacy import displacy
from spacy import tokenizer
import re
import pandas as pd

In [4]:
nlp = spacy.load('en_core_web_sm')

In [5]:
google_text = "Google was founded on September 4, 1998, by computer scientists Larry Page and Sergey Brin while they were PhD students at Stanford University in California. Together they own about 14% of its publicly listed shares and control 56% of its stockholder voting power through super-voting stock. The company went public via an initial public offering (IPO) in 2004. In 2015, Google was reorganized as a wholly owned subsidiary of Alphabet Inc. Google is Alphabet's largest subsidiary and is a holding company for Alphabet's internet properties and interests. Sundar Pichai was appointed CEO of Google on October 24, 2015, replacing Larry Page, who became the CEO of Alphabet. On December 3, 2019, Pichai also became the CEO of Alphabet."
print(google_text)

Google was founded on September 4, 1998, by computer scientists Larry Page and Sergey Brin while they were PhD students at Stanford University in California. Together they own about 14% of its publicly listed shares and control 56% of its stockholder voting power through super-voting stock. The company went public via an initial public offering (IPO) in 2004. In 2015, Google was reorganized as a wholly owned subsidiary of Alphabet Inc. Google is Alphabet's largest subsidiary and is a holding company for Alphabet's internet properties and interests. Sundar Pichai was appointed CEO of Google on October 24, 2015, replacing Larry Page, who became the CEO of Alphabet. On December 3, 2019, Pichai also became the CEO of Alphabet.


In [6]:
spacy_doc = nlp(google_text)

In [26]:
ner_df = pd.DataFrame(columns=['Word','NER'])

for word in spacy_doc.ents:
    row = {'Word':word.text, 'NER':word.label_}

    row_df = pd.DataFrame([row])

    ner_df = pd.concat([ner_df, row_df], ignore_index=True)

ner_df.head(10)

,Word,NER
0,Google,ORG
1,"September 4, 1998",DATE
2,Larry Page,PERSON
3,Sergey Brin,PERSON
4,PhD,WORK_OF_ART
5,Stanford University,ORG
6,California,GPE
7,about 14%,PERCENT
8,56%,PERCENT
9,IPO,ORG


In [28]:
google_text_clean = re.sub(r"[^\w\s]","",google_text).lower()
print(google_text_clean)

google was founded on september 4 1998 by computer scientists larry page and sergey brin while they were phd students at stanford university in california together they own about 14 of its publicly listed shares and control 56 of its stockholder voting power through supervoting stock the company went public via an initial public offering ipo in 2004 in 2015 google was reorganized as a wholly owned subsidiary of alphabet inc google is alphabets largest subsidiary and is a holding company for alphabets internet properties and interests sundar pichai was appointed ceo of google on october 24 2015 replacing larry page who became the ceo of alphabet on december 3 2019 pichai also became the ceo of alphabet


In [32]:
spacy_doc_clean = nlp(google_text_clean)

ner_df_clean = pd.DataFrame(columns=['Word','NER'])

for word in spacy_doc_clean.ents:
    row = {'Word':word.text, 'NER':word.label_}

    row_df = pd.DataFrame([row])

    ner_df_clean = pd.concat([ner_df_clean, row_df], ignore_index=True)

ner_df_clean.head(10)

,Word,NER
0,google,ORG
1,september 4 1998,DATE
2,stanford university,ORG
3,california,GPE
4,about 14,CARDINAL
5,56,CARDINAL
6,2004,DATE
7,2015,DATE
8,alphabet inc google,ORG
9,google,ORG


Google → ORG = Organization

September 4, 1998 → DATE = Date

Larry Page → PERSON = Person

PhD → WORK_OF_ART (sometimes misclassified, spaCy quirks)

Stanford University → ORG = Organization

California → GPE = Geopolitical Entity (state)

about 14% → PERCENT = Percentage

IPO → ORG (spaCy thought it’s an organization, but really it’s "Initial Public Offering")